In [4]:
%cd '/root/projects/nucleus_predictor'
import util.data
import util.data.transforms
import pandas as pd
import test_model
import model_modules.ttf_model
import os

/root/projects/nucleus_predictor


Create DataFrame from CZI and save the test set as CSV

In [3]:
path_czi = 'data/timelapse_czis/20160621_S01_001.czi'
path_save_csv = 'data/20160621_S01_001_test_set.csv'
name_signal = 'bf'
name_target = 'struct'
train_split = 0

df_train, df_test = util.data.create_train_test_dataframe_from_timelapse_czi(
        path_czi,
        name_signal=name_signal,
        name_target=name_target,
        train_split=train_split,
)
df_test.to_csv(path_save_csv, index=False)

reading: data/timelapse_czis/20160621_S01_001.czi


Build dataset from CSV and load model

In [5]:
df_test = pd.read_csv('data/20160621_S01_001_test_set.csv')
df_train = pd.DataFrame(columns=df_test.columns)  # empty train set

path_model = 'saved_models/ttf_lamin_b1_latest.p'
model = model_modules.ttf_model.Model(
    load_path=path_model,
    gpu_ids=0,
)
print('model:', model)


loading checkpoint from: saved_models/ttf_lamin_b1_latest.p
model load time: 2.4 s
model: ttf_v7_nn | iter: 50000


In [7]:
z_fac = 0.97
# xy_fac = 0.217  # timelapse czis; original scale 0.065 um/px
n_test_images = 8
save_images = False

xy_facs = [0.217]
xy_facs = [i/1000 for i in range(200, 300, 2)]
print(xy_facs)

dims_cropped = (32, '/16', '/16')
cropper = util.data.transforms.Cropper(dims_cropped, offsets=('mid', 0, 0))
transforms_provider = (cropper, cropper)
results = []
for xy_fac in xy_facs:
    path_save = os.path.join('tmp', 'xy_scale_{:.03f}'.format(xy_fac))
    resize_factors = (z_fac, xy_fac, xy_fac)
    resizer = util.data.transforms.Resizer(resize_factors)
    signal_transforms = (resizer, util.data.transforms.sub_mean_norm)
    target_transforms = (resizer, util.data.transforms.sub_mean_norm)
    transforms = (signal_transforms, target_transforms)
    ds = util.data.DataSet3(
        df_train=df_train,
        df_test=df_test,
        transforms=transforms,
    )
    ds.use_test_set()

    provider = util.data.TestImgDataProvider(
        dataset=ds,
        transforms=transforms_provider,
    )
    print('xy_fac:', xy_fac, path_save)
    kwargs = dict(
        n_images=n_test_images,
        path_save=path_save,
        save_images=save_images,
    )
    losses_dict = test_model.test_model(model, provider, **kwargs)
    losses_dict['xy_fac'] = xy_fac
    results.append(losses_dict)
    df_results = pd.DataFrame(results)
    df_results.to_csv('test_output/scale_check.csv')

[0.2, 0.202, 0.204, 0.206, 0.208, 0.21, 0.212, 0.214, 0.216, 0.218, 0.22, 0.222, 0.224, 0.226, 0.228, 0.23, 0.232, 0.234, 0.236, 0.238, 0.24, 0.242, 0.244, 0.246, 0.248, 0.25, 0.252, 0.254, 0.256, 0.258, 0.26, 0.262, 0.264, 0.266, 0.268, 0.27, 0.272, 0.274, 0.276, 0.278, 0.28, 0.282, 0.284, 0.286, 0.288, 0.29, 0.292, 0.294, 0.296, 0.298]
xy_fac: 0.2 tmp/xy_scale_0.20
reading: data/timelapse_czis/20160621_S01_001.czi


/opt/conda/envs/pytorch-py35/lib/python3.5/site-packages/scipy/ndimage/interpolation.py:568: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


DEBUG: cropper [slice(3, 35, None), slice(0, 192, None), slice(0, 192, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 192, None), slice(0, 192, None)]
test image: 00 | name: 20160621_S01_001.czi | l1: 0.7710 | l2: 1.2671 | l1_norm: 1.1919 | l2_norm: 1.0945
DEBUG: cropper [slice(3, 35, None), slice(0, 192, None), slice(0, 192, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 192, None), slice(0, 192, None)]
test image: 01 | name: 20160621_S01_001.czi | l1: 0.7711 | l2: 1.2378 | l1_norm: 1.1920 | l2_norm: 1.0723
DEBUG: cropper [slice(3, 35, None), slice(0, 192, None), slice(0, 192, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 192, None), slice(0, 192, None)]
test image: 02 | name: 20160621_S01_001.czi | l1: 0.7650 | l2: 1.2367 | l1_norm: 1.1852 | l2_norm: 1.0805
DEBUG: cropper [slice(3, 35, None), slice(0, 192, None), slice(0, 192, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 192, None), slice(0, 192, None)]
test image: 03 | name: 20160621_S01_001.czi | l1: 0.7664 |

DEBUG: cropper [slice(3, 35, None), slice(0, 208, None), slice(0, 208, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 208, None), slice(0, 208, None)]
test image: 03 | name: 20160621_S01_001.czi | l1: 0.7520 | l2: 1.2066 | l1_norm: 1.1561 | l2_norm: 1.0536
DEBUG: cropper [slice(3, 35, None), slice(0, 208, None), slice(0, 208, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 208, None), slice(0, 208, None)]
test image: 04 | name: 20160621_S01_001.czi | l1: 0.7672 | l2: 1.2525 | l1_norm: 1.1782 | l2_norm: 1.0962
DEBUG: cropper [slice(3, 35, None), slice(0, 208, None), slice(0, 208, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 208, None), slice(0, 208, None)]
test image: 05 | name: 20160621_S01_001.czi | l1: 0.7475 | l2: 1.1928 | l1_norm: 1.1510 | l2_norm: 1.0469
DEBUG: cropper [slice(3, 35, None), slice(0, 208, None), slice(0, 208, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 208, None), slice(0, 208, None)]
test image: 06 | name: 20160621_S01_001.czi | l1: 0.7577 |

DEBUG: cropper [slice(3, 35, None), slice(0, 208, None), slice(0, 208, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 208, None), slice(0, 208, None)]
test image: 06 | name: 20160621_S01_001.czi | l1: 0.7278 | l2: 1.1395 | l1_norm: 1.1345 | l2_norm: 1.0312
DEBUG: cropper [slice(3, 35, None), slice(0, 208, None), slice(0, 208, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 208, None), slice(0, 208, None)]
test image: 07 | name: 20160621_S01_001.czi | l1: 0.7214 | l2: 1.1259 | l1_norm: 1.1323 | l2_norm: 1.0280
*****
count: 8 | l1: 0.7288 | l2: 1.1610 | l1_norm: 1.1307 | l2_norm: 1.0266
Sorted by L2 norm loss
test 05 | L2 loss: 0.9760
test 03 | L2 loss: 0.9982
test 04 | L2 loss: 1.0231
test 07 | L2 loss: 1.0280
test 02 | L2 loss: 1.0292
test 06 | L2 loss: 1.0312
test 01 | L2 loss: 1.0582
test 00 | L2 loss: 1.0692
xy_fac: 0.214 tmp/xy_scale_0.21
reading: data/timelapse_czis/20160621_S01_001.czi
DEBUG: cropper [slice(3, 35, None), slice(0, 208, None), slice(0, 208, None)]
DEBUG: c

...

DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
test image: 03 | name: 20160621_S01_001.czi | l1: 0.6674 | l2: 1.0094 | l1_norm: 1.0237 | l2_norm: 0.8720
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
test image: 04 | name: 20160621_S01_001.czi | l1: 0.6789 | l2: 1.0298 | l1_norm: 1.0362 | l2_norm: 0.8929
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
test image: 05 | name: 20160621_S01_001.czi | l1: 0.6605 | l2: 0.9859 | l1_norm: 1.0135 | l2_norm: 0.8544
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
test image: 06 | name: 20160621_S01_001.czi | l1: 0.6674 |

DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
test image: 06 | name: 20160621_S01_001.czi | l1: 0.6669 | l2: 0.9929 | l1_norm: 1.0334 | l2_norm: 0.8901
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
test image: 07 | name: 20160621_S01_001.czi | l1: 0.6499 | l2: 0.9567 | l1_norm: 1.0152 | l2_norm: 0.8616
*****
count: 8 | l1: 0.6645 | l2: 0.9997 | l1_norm: 1.0279 | l2_norm: 0.8824
Sorted by L2 norm loss
test 07 | L2 loss: 0.8616
test 05 | L2 loss: 0.8684
test 00 | L2 loss: 0.8686
test 01 | L2 loss: 0.8851
test 03 | L2 loss: 0.8860
test 06 | L2 loss: 0.8901
test 04 | L2 loss: 0.8948
test 02 | L2 loss: 0.9042
xy_fac: 0.274 tmp/xy_scale_0.27
reading: data/timelapse_czis/20160621_S01_001.czi
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
DEBUG: c

DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
test image: 00 | name: 20160621_S01_001.czi | l1: 0.6551 | l2: 0.9883 | l1_norm: 1.0084 | l2_norm: 0.8552
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
test image: 01 | name: 20160621_S01_001.czi | l1: 0.6684 | l2: 1.0208 | l1_norm: 1.0266 | l2_norm: 0.8821
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
test image: 02 | name: 20160621_S01_001.czi | l1: 0.6716 | l2: 1.0296 | l1_norm: 1.0361 | l2_norm: 0.8962
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 272, None), slice(0, 272, None)]
test image: 03 | name: 20160621_S01_001.czi | l1: 0.6640 |

DEBUG: cropper [slice(3, 35, None), slice(0, 288, None), slice(0, 288, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 288, None), slice(0, 288, None)]
test image: 03 | name: 20160621_S01_001.czi | l1: 0.6711 | l2: 1.0399 | l1_norm: 1.0396 | l2_norm: 0.9105
DEBUG: cropper [slice(3, 35, None), slice(0, 288, None), slice(0, 288, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 288, None), slice(0, 288, None)]
test image: 04 | name: 20160621_S01_001.czi | l1: 0.6802 | l2: 1.0412 | l1_norm: 1.0463 | l2_norm: 0.9164
DEBUG: cropper [slice(3, 35, None), slice(0, 288, None), slice(0, 288, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 288, None), slice(0, 288, None)]
test image: 05 | name: 20160621_S01_001.czi | l1: 0.6697 | l2: 1.0178 | l1_norm: 1.0358 | l2_norm: 0.8982
DEBUG: cropper [slice(3, 35, None), slice(0, 288, None), slice(0, 288, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 288, None), slice(0, 288, None)]
test image: 06 | name: 20160621_S01_001.czi | l1: 0.6744 |

DEBUG: cropper [slice(3, 35, None), slice(0, 288, None), slice(0, 288, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 288, None), slice(0, 288, None)]
test image: 06 | name: 20160621_S01_001.czi | l1: 0.6771 | l2: 1.0404 | l1_norm: 1.0537 | l2_norm: 0.9421
DEBUG: cropper [slice(3, 35, None), slice(0, 288, None), slice(0, 288, None)]
DEBUG: cropper [slice(3, 35, None), slice(0, 288, None), slice(0, 288, None)]
test image: 07 | name: 20160621_S01_001.czi | l1: 0.6557 | l2: 0.9956 | l1_norm: 1.0285 | l2_norm: 0.9062
*****
count: 8 | l1: 0.6718 | l2: 1.0343 | l1_norm: 1.0411 | l2_norm: 0.9159
Sorted by L2 norm loss
test 01 | L2 loss: 0.8902
test 00 | L2 loss: 0.8986
test 05 | L2 loss: 0.9029
test 07 | L2 loss: 0.9062
test 02 | L2 loss: 0.9244
test 03 | L2 loss: 0.9304
test 04 | L2 loss: 0.9323
test 06 | L2 loss: 0.9421
xy_fac: 0.294 tmp/xy_scale_0.29
reading: data/timelapse_czis/20160621_S01_001.czi
DEBUG: cropper [slice(3, 35, None), slice(0, 288, None), slice(0, 288, None)]
DEBUG: c

In [11]:
df_results

,l1_norm_test,l1_test,l2_norm_test,l2_test,n_count_test,xy_fac
0,1.191643,0.766213,1.087693,1.229588,8,0.20
1,1.150783,0.742830,1.056149,1.194147,8,0.21
2,1.120400,0.730777,1.040369,1.198524,8,0.22
3,1.053860,0.679249,0.929717,1.053344,8,0.23
4,1.044492,0.675648,0.916871,1.040935,8,0.24
5,1.035285,0.676403,0.897386,1.038129,8,0.25
6,1.013785,0.653890,0.855539,0.966959,8,0.26
7,1.028167,0.667337,0.882541,1.007082,8,0.27
8,1.026678,0.662219,0.880673,0.995990,8,0.28
9,1.043030,0.673675,0.917953,1.037239,8,0.29
